#Google Drive Authetication



In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Installing Transformers

In [ ]:
!pip install transformers==3.0.0

     |████████████████████████████████| 754 kB 4.8 MB/s 
     |████████████████████████████████| 3.0 MB 20.2 MB/s 
     |████████████████████████████████| 895 kB 41.9 MB/s 
     |████████████████████████████████| 1.2 MB 35.8 MB/s 


#Importing Libraries

In [ ]:
#@title Importing Libraries

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.optim import lr_scheduler
from bs4 import BeautifulSoup 

import pandas as pd
import numpy as np
import os
import shutil

import logging
logging.basicConfig(level=logging.ERROR)

#import warnings
#warnings.filterwarnings("ignore")

#GPU device

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")


Device: cuda:0


#BERT Model

In [ ]:
class SentimentClassifier(nn.Module):
  """
  This class defines the model architecture which is simply a fully-connected
  layer on top of a pre-trained BERT model. 
  """

  def __init__(self, BERT_MODEL):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(BERT_MODEL)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, 3) # Number of output classes = 3

  def forward(self, ids, mask, token_type_ids):
    last_hidden_state, pooled_output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
labels_to_sentiment_dict = {
    0: 'positive',
    1: 'negative',
    2: 'neutral'
}
sentiment_to_labels_dict={
    'positive':0,
    'negative':1,
    'neutral':2

}

In [ ]:
import os
aspects = ["food", "staff", "miscellaneous", "place","service","menu","ambience","price"]

# models_dict:Is a dictionary containing models corresponding to all the 
# `location-aspect`.

models_dict = {}


for aspect in aspects:
  path = "/content/drive/MyDrive/NLU2/"+str(aspect)+"/"
  dir_list = os.listdir(path)
  path=path+dir_list[0]
  model = torch.load(path, map_location='cpu')
  model= model.to(device)
  models_dict[f"{aspect}"] = model

#Module 2

In [ ]:
file = open("/content/drive/MyDrive/1-restaurant-train.csv",'r')
contents = file.read()


In [ ]:
x = contents.split("\n")
print(len(x))
x=x[:10001]

82066


In [ ]:
text=[]
ratings=[]
id=[]
count=1
for i in range(len(x)-1):
  temp = x[i].split("\t")
  text.append(temp[1])
  ratings.append(temp[0])
  id.append(count)
  count=count+1
testing_set = pd.DataFrame(list(zip(id,text, ratings)),columns =['id','text', 'Ratings'])

In [ ]:
testing_set.head()

,id,text,Ratings
0,1,Thank you thank you thank you !! I want to th...,4
1,2,A Humane Society store at the Biltmore? Inter...,5
2,3,Don't buy Nike sneakers if you want to return ...,1
3,4,I have to say I love most things about Sprouts...,3
4,5,The tire pressure light came on a day or so ag...,5


In [ ]:
compressed_ratings = list(testing_set.Ratings[:10001])

In [ ]:
new_rat = [int(i) for i in compressed_ratings]

In [ ]:
import string, nltk
#nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
from bs4 import BeautifulSoup

lemmatizer = nltk.stem.WordNetLemmatizer()
def preprocess(text):
    text = str(text)
    soup = BeautifulSoup(text, "html.parser")
    for data in soup(['script', 'style']):
        data.decompose()
    text = ' '.join(soup.stripped_strings)
    #text = text.lower()
    temp = ""
    for i in text:
        if i in string.punctuation:
            continue
        else:
            temp+=i
    temp = temp.split()
    #temp = [w for w in temp if w not in nltk.corpus.stopwords.words('english')]
    for i in range(len(temp)):
        temp[i] = lemmatizer.lemmatize(temp[i])
    return ' '.join(temp)
testing_set['text'] = testing_set['text'].apply(preprocess)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
testing_set.head()

,id,text,Ratings
0,1,Thank you thank you thank you I want to thank ...,4
1,2,A Humane Society store at the Biltmore Interes...,5
2,3,Dont buy Nike sneaker if you want to return or...,1
3,4,I have to say I love most thing about Sprouts ...,3
4,5,The tire pressure light came on a day or so ag...,5


In [ ]:
len(testing_set)

10000

In [ ]:
BERT_MODEL = 'bert-base-uncased'
MAX_LEN = 160

tokenizer = transformers.BertTokenizer.from_pretrained(BERT_MODEL)
model_pred=[]
score=[]
aspect_category=[]
text_list=[]
test_id=[]
polarity=[]

count=0
for each_example in testing_set.itertuples():
  count+=1
  id = int(each_example.id)
  text_=each_example.text
  text = each_example.text.strip()
  each_example=list(each_example)


  for aspect in aspects:
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens = True,
        max_length = MAX_LEN,
        pad_to_max_length = True
    )
    ids = torch.tensor(inputs["input_ids"], dtype=torch.long).unsqueeze(0)
    mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).unsqueeze(0)
    token_type_ids = torch.tensor(inputs["token_type_ids"], dtype=torch.long).unsqueeze(0)

    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    token_type_ids = token_type_ids.to(device, dtype=torch.long)

    model = models_dict[f"{aspect}"]
    outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
 

    _, predicted = torch.max(outputs, 1)

    predicted = predicted.detach().cpu().numpy()
    predict_polarity=predicted[0]
    predict_score=outputs[0][predict_polarity].item()

    #append
    score.append(predict_score)
    test_id.append(id)
    aspect_category.append(aspect)
    text_list.append(text_)

    polarity.append(predicted[0])




In [ ]:
dict_ = {'id': test_id, 'text': text_list, 'aspect': aspect_category,'aspect_score':score,'polarity':polarity} 
df = pd.DataFrame(dict_)

In [ ]:
df.aspect_score = (df.aspect_score - np.min(df.aspect_score))/(np.max(df.aspect_score)-np.min(df.aspect_score))
print(df)

          id                                               text  \
0          1  Thank you thank you thank you I want to thank ...   
1          1  Thank you thank you thank you I want to thank ...   
2          1  Thank you thank you thank you I want to thank ...   
3          1  Thank you thank you thank you I want to thank ...   
4          1  Thank you thank you thank you I want to thank ...   
...      ...                                                ...   
79995  10000  I would have to say this is my absolute FAVORI...   
79996  10000  I would have to say this is my absolute FAVORI...   
79997  10000  I would have to say this is my absolute FAVORI...   
79998  10000  I would have to say this is my absolute FAVORI...   
79999  10000  I would have to say this is my absolute FAVORI...   

              aspect  aspect_score  polarity  
0               food      0.927050         0  
1              staff      0.292653         2  
2      miscellaneous      0.568492         0  
3      

In [ ]:
len(df)

80000

In [ ]:
'''
polarity_new=[]
for each_example in df.itertuples():
  if each_example.polarity==0:
    polarity_new.append("positive")
  elif each_example.polarity==1:
    polarity_new.append("negative")
  else:
    polarity_new.append("neutral")

df['polarity']=polarity_new'''
predicted_df = df
#predicted_df= df[df.aspect_score>=0.4]
print(predicted_df)

In [ ]:
predicted_df.to_csv("hello.csv",index=False)

NameError: ignored

In [ ]:
df=pd.read_csv("hello.csv")

In [ ]:
df.head(20)

,id,text,aspect,aspect_score,polarity
0,1,Thank you thank you thank you I want to thank ...,food,0.927050,0
1,1,Thank you thank you thank you I want to thank ...,staff,0.292653,2
2,1,Thank you thank you thank you I want to thank ...,miscellaneous,0.568492,0
3,1,Thank you thank you thank you I want to thank ...,place,0.463867,0
4,1,Thank you thank you thank you I want to thank ...,service,0.186245,2
5,1,Thank you thank you thank you I want to thank ...,menu,0.188491,1
6,1,Thank you thank you thank you I want to thank ...,ambience,0.457987,2
7,1,Thank you thank you thank you I want to thank ...,price,0.165106,2
8,2,A Humane Society store at the Biltmore Interes...,food,0.325218,2
9,2,A Humane Society store at the Biltmore Interes...,staff,0.578940,0


In [ ]:
aspects = ["food", "staff", "miscellaneous", "place","service","menu","ambience","price"]

In [ ]:

text=[]
food=[]
staff=[]
miscellaneous=[]
place=[]
service=[]
menu=[]
ambience=[]
price=[]
for i in range(0,len(df),8):
      text.append(df.text[i])
      food.append(df.polarity[i])
      staff.append(df.polarity[i+1])
      miscellaneous.append(df.polarity[i+2])
      place.append(df.polarity[i+3])
      service.append(df.polarity[i+4])
      menu.append(df.polarity[i+5])
      ambience.append(df.polarity[i+6])
      
      price.append(df.polarity[i+7])
   

new_df2 = pd.DataFrame(list(zip(text,food, staff,miscellaneous,place,service,menu,ambience,price,new_rat)),
                       columns =['text', "food", "staff", "miscellaneous", "place","service","menu","ambience","price","Ratings"])

In [ ]:
new_df2.head()

,text,food,staff,miscellaneous,place,service,menu,ambience,price,Ratings
0,Thank you thank you thank you I want to thank ...,0,2,0,0,2,1,2,2,4
1,A Humane Society store at the Biltmore Interes...,2,0,2,0,2,2,2,2,5
2,Dont buy Nike sneaker if you want to return or...,2,1,2,0,1,2,2,1,1
3,I have to say I love most thing about Sprouts ...,0,2,0,0,1,2,2,1,3
4,The tire pressure light came on a day or so ag...,2,1,2,2,2,2,2,2,5


In [ ]:
new_df2.to_csv("for_ppt.csv",index=False)

In [ ]:
len(new_df2)

10000

In [ ]:
y= new_df2.Ratings
X= new_df2.iloc[:,1:9]

In [ ]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
len(X_train)

In [ ]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy



#SVM

In [ ]:
model = svm.SVC()
svm_model = model.fit(X_train,y_train)

In [ ]:
X_test_pred = svm_model.predict(X_test)
X_train_pred = svm_model.predict(X_train)


In [ ]:
print("test Accuracy: ", accuracy(y_test, X_test_pred))
print("train Accuracy: ", accuracy(y_train, X_train_pred))

#Decision Tree

In [ ]:
clf = DecisionTreeClassifier(random_state=0)
tree_model = clf.fit(X_train,y_train)

In [ ]:
X_test_pred = tree_model.predict(X_test)
X_train_pred = tree_model.predict(X_train)

In [ ]:
print("test Accuracy: ", accuracy(y_test, X_test_pred))
print("train Accuracy: ", accuracy(y_train, X_train_pred))

#Random Forest

In [ ]:
clf_for = RandomForestClassifier(max_depth=10, random_state=0)


In [ ]:
clf_for.fit(X_train,y_train)

In [ ]:
X_test_pred = clf_for.predict(X_test)
X_train_pred = clf_for.predict(X_train)

In [ ]:
print("test Accuracy: ", accuracy(y_test, X_test_pred))
print("train Accuracy: ", accuracy(y_train, X_train_pred))

#Xgboost

In [ ]:
xgb_model = xgb.XGBRegressor()

In [ ]:
xgb_model.fit(X_train,y_train)

In [ ]:
X_test_pred = xgb_model.predict(X_test)
X_train_pred = xgb_model.predict(X_train)

In [ ]:
print("test Accuracy: ", accuracy(y_test, X_test_pred))
print("train Accuracy: ", accuracy(y_train, X_train_pred))